In [1]:
import opendatasets as od


od.download("https://www.kaggle.com/datasets/mostafaabla/garbage-classification")

Skipping, found downloaded files in "./garbage-classification" (use force=True to force download)


In [2]:
import torch
import sys
import platform
has_gpu = torch.cuda.is_available()
# has_mps = getattr(torch,'has_mps',False)
has_mps = torch.backends.mps.is_built()
device = "mps" if torch.backends.mps.is_built() \
    else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Python {sys.version}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AzVAILABLE")
print(f"Target device is {device}")

Python Platform: Linux-6.5.9-arch2-1-x86_64-with-glibc2.38
PyTorch Version: 2.1.1+cu121
Python 3.11.0 (main, Mar  1 2023, 18:26:19) [GCC 11.2.0]
NVIDIA/CUDA GPU is available
MPS (Apple Metal) is NOT AzVAILABLE
Target device is cuda


In [3]:
import pathlib
import numpy as np
dataPath = pathlib.Path.cwd() / "garbage-classification"

print(dataPath)

isFile = lambda path : path.is_file()

imagePaths = list(filter(isFile, list(dataPath.rglob("*"))))

pathSamples = np.random.choice(imagePaths, 1000)



/home/coldstuff1/codejam/CodeJam13/model/garbage-classification


In [4]:
from PIL import Image
import torchvision.transforms.v2 as v2

torch.cuda.empty_cache()

transformer = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.RandomResizedCrop(size=(256, 256), antialias=True),
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

inputImages = []

labelsList = []

rejected = []

for path in pathSamples:
    pathStr = str(path)
    labelNum = -1
    if 'battery' in pathStr:
        labelNum = 0
    elif 'biological' in pathStr:
        labelNum = 1
    elif 'brown-glass' in pathStr:
        labelNum = 2
    elif 'cardboard' in pathStr:
        labelNum = 3
    elif 'clothes' in pathStr:
        labelNum = 4
    elif 'green-glass' in pathStr:
        labelNum = 5
    elif 'metal' in pathStr:
        labelNum = 6
    elif 'paper' in pathStr:
        labelNum = 7
    elif 'plastic' in pathStr:
        labelNum = 8
    elif 'shoes' in pathStr:
        labelNum = 9
    elif 'trash' in pathStr:
        labelNum = 10
    elif 'white-glass' in pathStr:
        labelNum = 11
    if labelNum == -1:
        continue
    img = Image.open(pathStr)
    tensor = transformer(img).to(device)
    if(tensor.shape[0] != 3):
        rejected.append(pathStr)
        continue
    labelsList.append(labelNum)
    inputImages.append(tensor)
    


dataset = torch.stack(inputImages).to(device)
labels = torch.ByteTensor(labelsList)


print(dataset.shape)
print(labels.shape)



torch.Size([1000, 3, 256, 256])
torch.Size([1000])


In [5]:
train_size = int(0.8* len(dataset))
validate_size = int(0.1* len(dataset))
test_size = int(0.1* len(dataset))

print(train_size)
print(validate_size)
print(test_size)

800
100
100


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*62*62, 128)
        self.fc2 = nn.Linear(128, 12)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)   
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

torch.cuda.empty_cache()

for epoch in range(25):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i in range(0, train_size, 64):
        inputs, curLabels = dataset[i:i+64].to(device), labels[i:i+64].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, curLabels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / train_size}")

print("Finished Training")


Epoch 1, Loss: 0.035810856074094774
Epoch 2, Loss: 0.03183657348155975
Epoch 3, Loss: 0.029557222276926042
Epoch 4, Loss: 0.02763885796070099
Epoch 5, Loss: 0.025756762474775315
Epoch 6, Loss: 0.0240987403690815
Epoch 7, Loss: 0.02260261356830597
Epoch 8, Loss: 0.02102575808763504
Epoch 9, Loss: 0.01964906983077526
Epoch 10, Loss: 0.018355787992477418
Epoch 11, Loss: 0.016984615921974183
Epoch 12, Loss: 0.015516158416867256
Epoch 13, Loss: 0.014097649902105332
Epoch 14, Loss: 0.012682093530893326
Epoch 15, Loss: 0.011303008049726487
Epoch 16, Loss: 0.009971063062548637
Epoch 17, Loss: 0.008865347430109978
Epoch 18, Loss: 0.008002649135887623
Epoch 19, Loss: 0.007293289918452501
Epoch 20, Loss: 0.007047781925648451
Epoch 21, Loss: 0.006907192580401897
Epoch 22, Loss: 0.0074231156706809995
Epoch 23, Loss: 0.010239873826503754
Epoch 24, Loss: 0.012988388612866402
Epoch 25, Loss: 0.009158784002065658
Finished Training


In [11]:

# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(train_size, train_size+validate_size, 64):
        inputs, curLabels = dataset[index:index+64].to(device), labels[index:index+64].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the validation set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the validation set: 54.69%


In [12]:

# Initialize variables for tracking accuracy
correct = 0
total = 0

# Set the model to evaluation mode (important for models with dropout and batch normalization)
model.eval()

# Iterate through the test set and make predictions
with torch.no_grad():
    for index in range(train_size+validate_size, train_size+validate_size+test_size, 64):
        inputs, curLabels = dataset[index:index+64].to(device), labels[index:index+64].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += curLabels.size(0)
        correct += (predicted == curLabels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test_size set: {accuracy:.2f}%")# Load and preprocess the test set

Accuracy on the test_size set: 39.00%
